In [3]:
from TraGT.load_data import load_data, load_data_long,CustomDataset,adj_list_to_adj_matrix
import torch
from torch_geometric.data import Data
from TraGT.seq_models import TransformerModel
from TraGT.graph_models import Graph_Transformer
from TraGT_v2_recon.fusion_model import FusionModel
import torch.optim as optim
import torch.nn as nn
from datetime import datetime
import os
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

data_name='bbbp'

In [4]:
train_data, train_labels, test_data, test_labels = load_data_long(data_name, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

input_dim_train = train_data['features'][0].size(-1)
input_dim_test = test_data['features'][0].size(-1)


adj_matrices_train = [adj_list_to_adj_matrix(adj_list) for adj_list in train_data['adj_lists']]
adj_matrices_test = [adj_list_to_adj_matrix(adj_list) for adj_list in test_data['adj_lists']]


data_list_train = [Data(x=torch.tensor(features, dtype=torch.float),
                        edge_index=torch.nonzero(adj_matrix, as_tuple=False).t().contiguous(),
                        y=torch.tensor(label, dtype=torch.float))
                    for features, adj_matrix, label in zip(train_data['features'], adj_matrices_train, train_labels)]
data_list_test = [Data(x=torch.tensor(features, dtype=torch.float),
                                edge_index=torch.nonzero(adj_matrix, as_tuple=False).t().contiguous(),
                                y=torch.tensor(label, dtype=torch.float))
                            for features, adj_matrix, label in zip(test_data['features'], adj_matrices_test, test_labels)]

train_dataset = CustomDataset(data_list_train, train_data['sequence'])
test_dataset = CustomDataset(data_list_test, test_data['sequence'])
    

[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:32] WARNING: not removing hydrogen atom without neighbors
[15:44:33] WARNING: not removing hydrogen atom without neighbors
[15:44:33] WARNING: not removing hydrogen atom without neighbors
[15:44:33] WARNING: not r

In [6]:
import torch
vocab_size = 100
d_model = 100
nhead = 4
num_encoder_layers = 3
dim_feedforward = 512
max_length = 100
batch_size = 1
num_epochs = 100
model_path = 'saved_models/bbbp_2024-04-28_23-06-49/test_best_model_91.710.pth'

graph_model = Graph_Transformer(in_channels=input_dim_train, hidden_channels=64, out_channels=1, heads=4).to(device)

sequence_model = TransformerModel(vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward).to(device)

fusion_model = FusionModel(graph_model, sequence_model).to(device)

# Load the state dict into the model
fusion_model.load_state_dict(torch.load(model_path))


RuntimeError: Error(s) in loading state_dict for FusionModel:
	Unexpected key(s) in state_dict: "sequence_model.decoder.lstm.weight_ih_l0", "sequence_model.decoder.lstm.weight_hh_l0", "sequence_model.decoder.lstm.bias_ih_l0", "sequence_model.decoder.lstm.bias_hh_l0", "sequence_model.decoder.fc.weight", "sequence_model.decoder.fc.bias". 

In [ ]:
# Initialize lists to store outputs and labels
outputs_test = []
true_labels_test = []
pred_probs_test = []
total_correct = 0
total_samples = 0

for data_batch in test_dataset:
    graph_data_batch = data_batch[0]
    sequence_inputs = data_batch[1]
    sequence_targets = graph_data_batch.y
                
    output, reconstructed_sequence = fusion_model(graph_data_batch, sequence_inputs)
    binary_predictions = (output >= 0.5).float()
                
    batch_correct = (binary_predictions == sequence_targets).sum().item()
    total_correct += batch_correct
    total_samples += 1
                
    true_labels_test.append(sequence_targets.cpu().numpy().reshape(-1))
    pred_probs_test.append(output.detach().cpu().numpy()[0])
    outputs_test.append(output.detach().cpu().numpy())

# Convert lists to numpy arrays
outputs_test = np.concatenate(outputs_test, axis=0)
true_labels_test = np.concatenate(true_labels_test, axis=0)

# Use t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=0)
outputs_test_2d = tsne.fit_transform(outputs_test)

# Plot the results
plt.figure(figsize=(6, 5))
scatter = plt.scatter(outputs_test_2d[:, 0], outputs_test_2d[:, 1], c=true_labels_test)
plt.title('t-SNE plot of test data')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend(*scatter.legend_elements())
plt.show()

TypeError: 'collections.OrderedDict' object is not callable

In [ ]:
# Initialize lists to store outputs and labels
outputs_test = []
true_labels_test = []
pred_probs_test = []
total_correct = 0
total_samples = 0

for data_batch in test_dataset:
    graph_data_batch = data_batch[0]
    sequence_inputs = data_batch[1]
    sequence_targets = graph_data_batch.y
                
    output, reconstructed_sequence = fusion_model(graph_data_batch, sequence_inputs)
    binary_predictions = (output >= 0.5).float()
                
    batch_correct = (binary_predictions == sequence_targets).sum().item()
    total_correct += batch_correct
    total_samples += 1
                
    true_labels_test.append(sequence_targets.cpu().numpy().reshape(-1))
    pred_probs_test.append(output.detach().cpu().numpy()[0])
    outputs_test.append(output.detach().cpu().numpy())

# Convert lists to numpy arrays
outputs_test = np.concatenate(outputs_test, axis=0)
true_labels_test = np.concatenate(true_labels_test, axis=0)

# Use t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=0)
outputs_test_2d = tsne.fit_transform(outputs_test)

# Plot the results
plt.figure(figsize=(6, 5))
scatter = plt.scatter(outputs_test_2d[:, 0], outputs_test_2d[:, 1], c=true_labels_test)
plt.title('t-SNE plot of test data')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend(*scatter.legend_elements())
plt.show()

TypeError: 'collections.OrderedDict' object is not callable